In [1]:
import pandas as pd
import numpy as np
import json
import yfinance as yf
import scipy.stats as sp
from datetime import datetime
import urllib.request
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)

In [2]:
def get_option(ticker,date):
    unix = int((date-datetime(1970,1,1)).total_seconds())
    url = "https://query2.finance.yahoo.com/v7/finance/options/{}?date={}".format(ticker,unix)
    chain = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))
    quote = pd.Series(chain['optionChain']['result'][0]['quote'])
    calls = pd.DataFrame(chain['optionChain']['result'][0]['options'][0]['calls'])
    calls['type'] = 'c'
    puts =  pd.DataFrame(chain['optionChain']['result'][0]['options'][0]['puts'])
    puts['type'] = 'p'
    
    #function to format the frame
    def format_options(df):
        #time we got data
        t0 = quote['regularMarketTime']
        cols = ['contractSymbol','type','strike','expiration','openInterest',
                'volume','bid','ask']
        temp = df[cols].copy()
        temp.columns = ['Contract','Type','Strike','Expiration','OI',
                        'Volume','Bid','Ask']
        temp.insert(loc=0,column='Stock',value=ticker)
        temp.insert(loc=3,column='Underlying',value=quote['regularMarketPrice'])
        temp.insert(loc=6,column='TTM',
                    value=(temp['Expiration'].values-t0)/(3600*24*365))
        temp['Price'] = (temp['Bid']+temp['Ask'])/2
        temp['Moneyness'] = np.abs(temp['Underlying']-temp['Strike'])
        return(temp)
    return format_options(calls),format_options(puts),quote['regularMarketPrice']

In [3]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

def bisection(f, a, b, tol=1e-6):
    c = (a+b)/2
    while(np.abs(b-a)>tol):
        if f(c) == 0:
            return c
        elif f(a)*f(c)<0:
            b=c
        else:
            a=c
        c = (a+b)/2
    return c

def impliedVol(S0, K, tau, r, price, opt_type='c'):
    equation = lambda sigma: BSM(S0,K,tau,r,sigma,opt_type) - price
    return bisection(equation,-1,2)

In [4]:
# #data download
# stock ='AMZN'
date1 = datetime(2020,4,17)
date2 = datetime(2020,5,15)
date3 = datetime(2020,6,19)
# call1,put1,quote1 = get_option(stock,date1)
# call2,put2,quote2 = get_option(stock,date2)
# call3,put3,quote3 = get_option(stock,date3)

In [5]:
# # Reformatting Expiration Dates
# call1['Expiration'] = date1
# put1['Expiration'] = date1
# call2['Expiration'] = date2
# put2['Expiration'] = date2
# call3['Expiration'] = date3
# put3['Expiration'] = date3

In [6]:
# # Writing to csv
# call1.to_csv('AMZN_Calls1.csv')
# put1.to_csv('AMZN_Puts1.csv')
# call2.to_csv('AMZN_Calls2.csv')
# put2.to_csv('AMZN_Puts2.csv')
# call3.to_csv('AMZN_Calls3.csv')
# put3.to_csv('AMZN_Puts3.csv')

In [7]:
call1 = pd.read_csv('AMZN_Calls1.csv')
call1.drop('Unnamed: 0',axis=1,inplace=True)
call1.dropna(axis = 0, inplace = True)
call1 = call1.nsmallest(20,'Moneyness').sort_values('Strike')
put1 = pd.read_csv('AMZN_Puts1.csv')
put1.drop('Unnamed: 0',axis=1,inplace=True)
put1.dropna(axis = 0, inplace = True)
put1 = put1.nsmallest(20,'Moneyness').sort_values('Strike')
call2 = pd.read_csv('AMZN_Calls2.csv')
call2.drop('Unnamed: 0',axis=1,inplace=True)
call2.dropna(axis = 0, inplace = True)
call2 = call2.nsmallest(20,'Moneyness').sort_values('Strike')
put2 = pd.read_csv('AMZN_Puts2.csv')
put2.drop('Unnamed: 0',axis=1,inplace=True)
put2.dropna(axis = 0, inplace = True)
put2 = put2.nsmallest(20,'Moneyness').sort_values('Strike')
call3 = pd.read_csv('AMZN_Calls3.csv')
call3.drop('Unnamed: 0',axis=1,inplace=True)
call3.dropna(axis = 0, inplace = True)
call3 = call3.nsmallest(20,'Moneyness').sort_values('Strike')
put3 = pd.read_csv('AMZN_Puts3.csv')
put3.drop('Unnamed: 0',axis=1,inplace=True)
put3.dropna(axis = 0, inplace = True)
put3 = put3.nsmallest(20,'Moneyness').sort_values('Strike')

# Problem 1: The Binomial Tree

In [8]:
def binomial_tree(S, K, T, r, sig, N, opt_type='c', am=False):
    if opt_type == 'c': #c is multiplied by payout s - k so that it is negated to k - s for puts
        c = 1
    else:
        c = -1
    dt = T/N
    v = r-.5*sig**2
    u = np.sqrt((v*dt)**2 + dt*sig**2)
    d = -u
    pu = .5+.5*(v*dt)/u
    pd = 1-pu
    disc = np.exp(-r*dt)
    
    #initialize stock and option paths
    stock_price = np.zeros((N+1,N+1))
    opt_val = np.zeros((N+1,N+1))
    
    #set stock price to S at time 0
    stock_price[0,0] = S
    
    #stock price generation, first initialize down paths
    for i in range(1,N+1):
        stock_price[i,0] = stock_price[i-1,0]*np.exp(d)
        # for each down path assign up paths
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*np.exp(u)
    
    #terminal option values derived from final stock prices in each path
    opt_val[N,:] = np.maximum(0,c*(stock_price[N,:]-K))
    
    #Recursion For Option Price
    #similar to stock paths but working backwards to calculate option values until time 0 is reached
    for i in range(N-1,-1,-1):
        for j in range(i+1):
            opt_val[i,j] = disc*(pu*opt_val[i+1,j+1]+
                                 pd*opt_val[i+1,j])
            if am: #adding exercise optionality for american options
                opt_val[i,j] = max(opt_val[i,j],
                                   c*(stock_price[i,j]-K))
    return opt_val[0,0]

In [9]:
temp = [call1,put1,call2,put2,call3,put3]
opts = pd.concat(temp)
opts.reset_index(inplace = True, drop = True)
opts['Expiration'] = pd.to_datetime(opts['Expiration'])

In [12]:
# Calculating Implied Vol and Black-Scholes Price
r = .0159 # Fed Funds Rate
opts['IV'] = opts.apply(lambda x: impliedVol(x['Underlying'], x['Strike'], x['TTM'], r, x['Price'], x['Type']), axis = 1)
opts['BS_Price'] = opts.apply(lambda x: BSM(x['Underlying'], x['Strike'], x['TTM'], r, x['IV'], x['Type']), axis = 1)

In [13]:
# Calculating Price with Binomial Tree
opts['Euro_Price'] = opts.apply(lambda x: binomial_tree(x['Underlying'], x['Strike'], x['TTM'], r, x['IV'], 200, 
                                                        x['Type']), axis = 1)
opts['Am_Price'] = opts.apply(lambda x: binomial_tree(x['Underlying'], x['Strike'], x['TTM'], r, x['IV'], 200, 
                                                        x['Type'], True), axis = 1)

In [59]:
# Calls
for dat in [date1, date2, date3]:
    temp = opts[(opts['Expiration'] == dat) & (opts['Type'] == 'c')]
    trace0 = go.Scatter(
        x = temp['Strike'], y = temp['Am_Price'], name = 'American'
    )
    trace1 = go.Scatter(
        x = temp['Strike'], y = temp['Euro_Price'], name = 'European'
    )
    trace2 = go.Scatter(
        x = temp['Strike'], y = temp['BS_Price'], name = 'Black-Scholes'
    )
    trace3 = go.Scatter(
        x = temp['Strike'], y = temp['Bid'], name = 'Bid'
    )
    trace4 = go.Scatter(
        x = temp['Strike'], y = temp['Ask'], name = 'Ask'
    )
    trace5 = go.Scatter(
        x = temp['Underlying'], y = np.linspace(min(temp['Bid']) ,max(temp['Ask']) ,20), name = 'Underlying', 
    )
    data = [trace0, trace1, trace2, trace3, trace4, trace5]
    layout = go.Layout(
    title = 'AMZN Call Price vs Strike Expiration: {}'.format(dat), yaxis = dict(title = 'Price'), 
        xaxis = dict(title = 'Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
    
    
    
    

In [60]:
# Puts
for dat in [date1, date2, date3]:
    temp = opts[(opts['Expiration'] == dat) & (opts['Type'] == 'p')]
    trace0 = go.Scatter(
        x = temp['Strike'], y = temp['Am_Price'], name = 'American'
    )
    trace1 = go.Scatter(
        x = temp['Strike'], y = temp['Euro_Price'], name = 'European'
    )
    trace2 = go.Scatter(
        x = temp['Strike'], y = temp['BS_Price'], name = 'Black-Scholes'
    )
    trace3 = go.Scatter(
        x = temp['Strike'], y = temp['Bid'], name = 'Bid'
    )
    trace4 = go.Scatter(
        x = temp['Strike'], y = temp['Ask'], name = 'Ask'
    )
    trace5 = go.Scatter(
        x = temp['Underlying'], y = np.linspace(min(temp['Bid']) ,max(temp['Ask']) ,20), name = 'Underlying', 
    )
    data = [trace0, trace1, trace2, trace3, trace4, trace5]
    layout = go.Layout(
    title = 'AMZN Put Price vs Strike Expiration: {}'.format(dat), yaxis = dict(title = 'Price'), 
        xaxis = dict(title = 'Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [65]:
opts[['Stock', 'Type', 'Underlying', 'Strike', 'Expiration', 'Volume', 'Bid', 'Ask', 'Price', 'IV', 'BS_Price', 
      'Euro_Price', 'Am_Price']].sample(20)

,Stock,Type,Underlying,Strike,Expiration,Volume,Bid,Ask,Price,IV,BS_Price,Euro_Price,Am_Price
60,AMZN,p,1932.5763,1780.0,2020-05-15,11.0,65.00,67.20,66.100,0.401859,66.099969,66.186310,66.333640
25,AMZN,p,1932.5763,1900.0,2020-04-17,147.0,75.70,77.60,76.650,0.360307,76.650037,76.552143,76.717468
26,AMZN,p,1932.5763,1905.0,2020-04-17,4.0,78.35,79.10,78.725,0.359305,78.725043,78.739622,78.908574
95,AMZN,c,1932.5763,1960.0,2020-06-19,5.0,132.70,134.55,133.625,0.343422,133.625080,133.801195,133.801195
11,AMZN,c,1932.5763,1930.0,2020-04-17,10.0,101.10,103.20,102.150,0.378443,102.150008,102.093974,102.093974
75,AMZN,p,1932.5763,2040.0,2020-05-15,4.0,174.05,177.35,175.700,0.341144,175.699968,175.829988,176.492437
106,AMZN,p,1932.5763,1915.0,2020-06-19,2.0,128.00,130.30,129.150,0.345651,129.149932,129.294324,129.821516
87,AMZN,c,1932.5763,1920.0,2020-06-19,1.0,153.90,156.40,155.150,0.350783,155.150083,155.237290,155.237290
7,AMZN,c,1932.5763,1910.0,2020-04-17,6.0,113.85,115.40,114.625,0.388065,114.624956,114.734326,114.734326
66,AMZN,p,1932.5763,1900.0,2020-05-15,10.0,106.15,107.95,107.050,0.374588,107.050113,107.139816,107.443971


The plots and summary table are consistent in showing the American and European option prices calculated through the binary tree are very similar to the values yielded from the Black-Scholes formula. As expected in all instances for call options the European price is equivalent to the American price because Amazon is a nondividend paying such and as such it is never optimal to exercise early for a call option. Similarly all put options yield a higher price for American options than European due to the option to exercise early. In each plot, as expected, all three prices are inside the bid ask spread, that is greater than the bid and less than the ask. The plots of the puts exhibit a positive relationship between the strike and option price whereas the calls display a negative relationship where option prices fall as the strike rises since there is a lower likelihood that the option will be exercised.

In [37]:
# Computing European Put Prices with Binomial Tree and Varying N
N_s = [10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 400]
cols = ['Stock', 'Type', 'Underlying', 'Strike', 'Expiration', 'TTM', 'IV', 'BS_Price']
euro_puts = opts[cols][opts['Type'] == 'p']
for n in N_s:
    euro_puts['N={}'.format(n)] = euro_puts.apply(lambda x: binomial_tree(x['Underlying'], x['Strike'], x['TTM'],r,
                                                                 x['IV'], n, x['Type']), axis = 1)
euro_puts.head()

,Stock,Type,Underlying,Strike,Expiration,TTM,IV,BS_Price,N=10,N=20,N=30,N=40,N=50,N=100,N=150,N=200,N=250,N=300,N=350,N=400
20,AMZN,p,1932.5763,1875.0,2020-04-17,0.115785,0.381260,70.850003,72.836480,72.032456,71.526553,71.181700,70.927967,70.869284,70.999625,70.947354,70.855057,70.781361,70.861259,70.894955
21,AMZN,p,1932.5763,1880.0,2020-04-17,0.115785,0.382734,73.374916,75.186796,74.562785,74.137928,73.841646,73.621143,73.237389,73.479572,73.494106,73.447407,73.383210,73.315786,73.363933
22,AMZN,p,1932.5763,1885.0,2020-04-17,0.115785,0.380573,75.024980,76.624640,76.169343,75.821719,75.572092,75.383666,74.853281,75.050370,75.125252,75.120254,75.087096,75.043939,74.998041
23,AMZN,p,1932.5763,1890.0,2020-04-17,0.115785,0.378866,76.825082,78.175230,77.890049,77.620132,77.417398,77.261191,76.811430,76.733506,76.869536,76.906723,76.904926,76.886269,76.860197
24,AMZN,p,1932.5763,1895.0,2020-04-17,0.115785,0.375157,78.150024,79.219676,79.098355,78.904036,78.747137,78.622513,78.252096,78.060295,78.105949,78.183161,78.211283,78.216091,78.209067


In [93]:
err = euro_puts.iloc[:,8:20]
err = err.apply(lambda x: abs(x - euro_puts['BS_Price']), axis = 0)
err['Expiration'] = euro_puts['Expiration']
temp = err.groupby('Expiration').mean()
trace0 = go.Scatter(
    x = N_s, y = temp.iloc[0,:], name = str(date1)[0:10]
)
trace1 = go.Scatter(
    x = N_s, y = temp.iloc[1,:], name = str(date2)[0:10]
)
trace2 = go.Scatter(
    x = N_s, y = temp.iloc[2,:], name = str(date3)[0:10]
)
data = [trace0, trace1, trace2]
layout = go.Layout(
    title = 'Absolute Error vs Number of Steps', yaxis = dict(title = 'Error'), 
        xaxis = dict(title = 'Steps (N)'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

For the most part, the error is smaller for shorter maturities and the error decreases exponentially as the number of steps increases. It is also worth noting that between 250 and 300 time steps the plots of all three expirations converge to essentially the same error around 5bps of the Black-Scholes price.

# Problem 2: Implied Volatility

In [23]:
#S, K, T, r, sig, N, opt_type='c', am=False
def impliedVol_tree(S0, K, tau, r, price, opt_type='c', am=False):
    equation = lambda sigma: binomial_tree(S0,K,tau,r,sigma,
                                          150,opt_type,am) - price
    return bisection(equation,0,1)

opts['IV_tree'] = opts.apply(lambda x: impliedVol_tree(x['Underlying'], x['Strike'], x['TTM'], r, 
                                                       x['Price'], x['Type']), axis = 1)
opts['IV_tree_AM'] = opts.apply(lambda x: impliedVol_tree(x['Underlying'], x['Strike'], x['TTM'], r, 
                                                       x['Price'], x['Type'], True), axis = 1)

In [28]:
opts[['Stock', 'Type', 'Underlying', 'Strike', 'Expiration', 'Volume', 'Price', 'IV', 'IV_tree', 'IV_tree_AM',
      'BS_Price', 'Euro_Price', 'Am_Price']].sample(10)

,Stock,Type,Underlying,Strike,Expiration,Volume,Price,IV,IV_tree,IV_tree_AM,BS_Price,Euro_Price,Am_Price
97,AMZN,c,1932.5763,1970.0,2020-06-19,13.0,129.175,0.343190,0.342703,0.342703,129.175114,129.274457,129.274457
59,AMZN,c,1932.5763,2090.0,2020-05-15,1.0,59.350,0.342054,0.341970,0.341970,59.349999,59.433755,59.433755
68,AMZN,p,1932.5763,1940.0,2020-05-15,12.0,120.300,0.353640,0.353623,0.352458,120.300022,120.323357,120.709038
74,AMZN,p,1932.5763,2030.0,2020-05-15,16.0,169.950,0.343519,0.344090,0.342092,169.949960,170.042922,170.674512
1,AMZN,c,1932.5763,1880.0,2020-04-17,3.0,133.725,0.399880,0.399579,0.399579,133.725040,133.845386,133.845386
73,AMZN,p,1932.5763,2020.0,2020-05-15,11.0,164.550,0.346547,0.346349,0.344576,164.550054,164.463836,165.070139
11,AMZN,c,1932.5763,1930.0,2020-04-17,10.0,102.150,0.378443,0.378772,0.378772,102.150008,102.093974,102.093974
15,AMZN,c,1932.5763,1950.0,2020-04-17,96.0,91.025,0.371665,0.371066,0.371066,91.025087,91.147198,91.147198
95,AMZN,c,1932.5763,1960.0,2020-06-19,5.0,133.625,0.343422,0.342852,0.342852,133.625080,133.801195,133.801195
105,AMZN,p,1932.5763,1910.0,2020-06-19,5.0,127.250,0.347154,0.346633,0.345365,127.249944,127.423273,127.938102


In [26]:
temp = opts[(opts['Type'] == 'p') & (opts['Expiration'] == date1)]
trace0 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree'])/temp['IV']*100, name = 'Euro IV {}'.format(date1)
)
trace1 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree_AM'])/temp['IV']*100, name = 'American IV {}'.format(date1)
)

temp = opts[(opts['Type'] == 'p') & (opts['Expiration'] == date2)]
trace2 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree'])/temp['IV']*100, name = 'Euro IV {}'.format(date2)
)
trace3 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree_AM'])/temp['IV']*100, name = 'American IV {}'.format(date2)
)

temp = opts[(opts['Type'] == 'p') & (opts['Expiration'] == date3)]
trace4 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree'])/temp['IV']*100, name = 'Euro IV {}'.format(date3)
)
trace5 = go.Scatter(
    x = temp['Strike'], y = abs(temp['IV'] - temp['IV_tree_AM'])/temp['IV']*100, name = 'American IV {}'.format(date3)
)
data = [trace0, trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    title = 'Binomial Tree Implied Vol vs BSM', yaxis = dict(title = 'Error (%)'), 
        xaxis = dict(title = 'Strike'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

All binary tree calculated implied volailities are within 10bps of the Black-Scholes implied volatility except the put options that are treated as American. These were still within 1% of the Black-Scholes values for all expirations. As seen before, the American and European calculations for call options are identical as Amazon does not pay a dividend thus early exercise is never optimal for calls. There is no clear relationship between the error and strike price as the error is small and similar for all strike prices. Similarly, there is also not a very significant relationship between error and time to maturity except for the American puts, which show slightly higher error as the time to maturity increases. 

# Problem 3: Trinomial Tree

In [30]:
def trinomial_tree(S, K, T, r, sig, q=0, N=200, opt_type='c', am=False):
    if opt_type == 'c':
        c = 1 
    else:
        c = -1
    dt = T/N
    dx = sig*np.sqrt(3*dt)
    v = r-q-.5*sig**2
    pu = .5*((sig**2*dt+v**2*dt**2)/dx**2 + v*dt/dx)
    pm = 1 - (sig**2*dt+v**2*dt**2)/dx**2
    pd = .5*((sig**2*dt+v**2*dt**2)/dx**2 - v*dt/dx)
    disc = np.exp(-r*dt)
    
    #initialize stock and option path matrices
    stock = np.zeros((N+1,2*N+1))
    option = np.zeros((N+1,2*N+1))
    
    #set root of tree to underlying quote
    stock[0,N]=S
    
    #loop to fill stock price matrix
    for i in range(1,N+1):
            stock[i,N-i:N+i+1] = [S*np.exp(dx*j) for j in range(-i,i+1)]
    
    #calculate and initialize final option values
    option[N:] = np.maximum(0,c*(stock[N,:]-K))
    #loop through to propogate option values
    for i in range(N-1,-1,-1):
        for j in range(N-i,N+i+1):
            option[i,j] = disc*(pd*option[i+1,j-1]+pm*option[i+1,j]+
                                pu*option[i+1,j+1])
            if am: #calculating whether or not to exercise early for american options
                option[i,j] = max(option[i,j],max(0,c*(stock[i,j]-K)))
    return option[0,N]
trinomial_tree(100, 100, 1, .05, .2, .02)

9.21751859646908

In [45]:
s = 100
k = 100
ttm = 1
r = .05
sig = .2
q = .02


TriTree_df = pd.DataFrame(columns = ['BS_Call', 'BS_Put', 'BTree_Call_Euro', 'BTree_Put_Euro', 
                                     'BTree_Call_Am', 'BTree_Put_Am', 'TTree_Call_Euro', 'TTree_Put_Euro', 
                                    'TTree_Call_Am', 'TTree_Put_Am'], index = N_s)
for n in N_s:
    temp = []
    temp.append(BSM(s, k, ttm, r, sig, 'c', q))
    temp.append(BSM(s, k, ttm, r, sig, 'p', q))
    temp.append(binomial_tree(s, k, ttm, r, sig, n, 'c'))
    temp.append(binomial_tree(s, k, ttm, r, sig, n, 'p'))
    temp.append(binomial_tree(s, k, ttm, r, sig, n, 'c', True))
    temp.append(binomial_tree(s, k, ttm, r, sig, n, 'p', True))
    temp.append(trinomial_tree(s, k, ttm, r, sig, q, n, 'c'))
    temp.append(trinomial_tree(s, k, ttm, r, sig, q, n, 'p'))
    temp.append(trinomial_tree(s, k, ttm, r, sig, q, n, 'c', True))
    temp.append(trinomial_tree(s, k, ttm, r, sig, q, n, 'p', True))
    TriTree_df.loc[n] = temp
    
TriTree_df


,BS_Call,BS_Put,BTree_Call_Euro,BTree_Put_Euro,BTree_Call_Am,BTree_Put_Am,TTree_Call_Euro,TTree_Put_Euro,TTree_Call_Am,TTree_Put_Am
10,9.22701,6.33008,10.2583,5.38653,10.2583,6.01408,9.03211,6.13519,9.03211,6.46862
20,9.22701,6.33008,10.3537,5.47934,10.3537,6.05215,9.13102,6.23409,9.13102,6.56516
30,9.22701,6.33008,10.3859,5.51059,10.3859,6.06568,9.1633,6.26638,9.1633,6.59682
40,9.22701,6.33008,10.402,5.52628,10.402,6.0722,9.17933,6.28241,9.17933,6.61501
50,9.22701,6.33008,10.4117,5.5357,10.4117,6.0757,9.18891,6.29199,9.18891,6.62305
100,9.22701,6.33008,10.4311,5.55459,10.4311,6.08334,9.20801,6.31108,9.20801,6.64219
150,9.22701,6.33008,10.4376,5.5609,10.4376,6.0857,9.21435,6.31743,9.21435,6.64854
200,9.22701,6.33008,10.4408,5.56405,10.4408,6.08688,9.21752,6.32059,9.21752,6.65157
250,9.22701,6.33008,10.4428,5.56595,10.4428,6.08757,9.21942,6.32249,9.21942,6.6534
300,9.22701,6.33008,10.4441,5.56721,10.4441,6.08805,9.22068,6.32376,9.22068,6.65463


The binary tree calculated price is significantly different from the Black-Scholes and trinomial tree prices and I have been unable to find my mistake that is responsible for this discrepency. The trinomial prices behave as expected, converging to the Black-Scholes price as the number of time steps increases.

In [53]:
trace0 = go.Scatter(
    x = TriTree_df.index, y = 100*abs(TriTree_df['TTree_Put_Euro'] - TriTree_df['BS_Put'])/TriTree_df['BS_Put']
)
data = [trace0]
layout = go.Layout(
    title = 'Trinomial Tree Error vs Number of Time Steps', yaxis = dict(title = 'Error (%)'), 
        xaxis = dict(title = 'Time Steps'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

With an n greater than 30 the error falls below 1% so the Trinomial Tree calculated price is within 1% of the Black-Scholes price, and as n reaches 400 that error falls to less than 10bps of the Black-Scholes price. The shape of the curve is very similar to the error curve of the binary tree for the AMZN European Puts with convergence at similar points.

In [110]:
def up_and_out(S, K, T, r, sig, N, H, opt_type='c', am=False):
    if opt_type == 'c': #c is multiplied by payout s - k so that it is negated to k - s for puts
        c = 1
    else:
        c = -1
    dt = T/N
    v = r-.5*sig**2
    u = np.sqrt((v*dt)**2 + dt*sig**2)
    d = -u
    pu = .5+.5*(v*dt)/u
    pd = 1-pu
    disc = np.exp(-r*dt)
    
    #initialize stock and option paths
    stock_price = np.zeros((N+1,N+1))
    opt_val = np.zeros((N+1,N+1))
    
    #set stock price to S at time 0
    stock_price[0,0] = S
    
    #stock price generation, first initialize down paths
    for i in range(1,N+1):
        stock_price[i,0] = stock_price[i-1,0]*np.exp(d)
        # for each down path assign up paths
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*np.exp(u)
    
    #terminal option values derived from final stock prices in each path
    #option value is 0 if the stock price is greater than the boundary
    for n in range(0,N):
        if stock_price[N,n] > H:
            opt_val[N,n] = 0
        else:
            opt_val[N,n] = np.maximum(0,c*(stock_price[N,n]-K))
    #Recursion For Option Price
    #similar to stock paths but working backwards to calculate option values until time 0 is reached
    for i in range(N-1,-1,-1):
        for j in range(i+1):
            if stock_price[i,j] < H: #again checking if the stock price is above the boundary
                if am: #adding exercise optionality for american options
                    opt_val[i,j] = max(opt_val[i,j],
                                       c*(stock_price[i,j]-K))
                else:
                    opt_val[i,j] = disc*(pu*opt_val[i+1,j+1]+
                                         pd*opt_val[i+1,j])
            else:
                opt_val[i,j] = 0
    return opt_val[0,0]

In [70]:
'Price of European Up and Out Call option with S = 10, K = 10, Tau = .3, r = 0.01, sigma = 0.2, H = 11, and 200 time steps: ${}'.format(round(up_and_out(10,10,.3,.01,.2,200,11),5))

'Price of European Up and Out Call option with S = 10, K = 10, Tau = .3, r = 0.01, sigma = 0.2, H = 11, and 200 time steps: $0.06262'

In [75]:
def d(S, K, r, T, sigma, q=0):
    v = r - q - .5*sigma**2
    return (np.log(S/K) + v*T)/(sigma*np.sqrt(T))
def UOanalytical(S, K, r, T, sig, H, q=0):
    disc = np.exp(-r*T)
    BS_SK = BSM(S,K,T,r,sig)
    BS_SH = BSM(S,H,T,r,sig)
    dBS1 = d(S,H,r,T,sig)
    v = r - q - .5*sigma**2
    BS_3 = BSM((H**2)/S,K,T,r,sig)
    BS_4 = BSM((H**2)/S,H,T,r,sig)
    dBS2 = d(H,S,r,T,sig)
    N = lambda x: sp.norm.cdf(x)
    UO = BS_SK - BS_SH - (H-K)*disc*N(dBS1) - ((H/S)**(2*v/(sig**2)))*(BS_3 - BS_4 - (H-K)*disc*N(dBS2))
    return UO

In [78]:
s = 10
k = 10
tau = .3
r = .01
sigma = .2
h = 11
'Binomial Tree Approximation (N=200): ${}, Analytical Solution: ${}'.format(round(up_and_out(10,10,.3,.01,.2,200,11),5), round(UOanalytical(s,k,r,tau,sigma,h), 5))

'Binomial Tree Approximation: $0.06262, Analytical Solution: $0.05309'

In [80]:
'Binomial Tree Approximation (N=1000): ${}, N=2000: ${}'.format(round(up_and_out(10,10,.3,.01,.2,1000,11),5), round(up_and_out(10,10,.3,.01,.2,2000,11),5))

'Binomial Tree Approximation (N=1000): $0.05578, N=2000: $0.05344'

The approximation is consistent with the analytical; however a larger number of time steps is required for precision with the approximation using 200 steps as I have throughout this assignment is relatively inaccurate. Using N=1000 and N=2000 yields solutions that are much closer to the analytical price with the 2000 step approximation being within 1% of the analytical solution with a deviation of only $0.000352 or a few hundreths of one cent.

In [89]:
def UIanalytical(S, K, r, T, sig, H, q=0):
    v = r - q - .5*sigma**2
    P1 = BSM((H**2)/S,K,T,r,sig,'p',q)
    P2 = BSM((H**2)/S,H,T,r,sig,'p',q)
    N = lambda x: sp.norm.cdf(x)
    disc = np.exp(-r*T)
    dBS1 = d(H,S,r,T,sig)
    C1 = BSM(S,H,T,r,sig,'c',q)
    dBS2 = d(S,H,r,T,sig)
    UI = (H/S)**(2*v/(sig**2))*(P1 - P2 + (H-K)*disc*N(-dBS1)) + C1 + (H-K)*disc*N(dBS2)
    return UI

In [102]:
UI = BSM(s,k,tau,r,sigma) - UOanalytical(s,k,r,tau,sigma,h)
'The analytically calculated price for the European Up and Out Call option is ${}, and the price via the In-Out Parity is ${}'.format(round(UIanalytical(s,k,r,tau,sigma,h),5),round(UI,5) )

'The analytically calculated price for the European Up and Out Call option is $0.39819, and the price via the In-Out Parity is $0.39819'

In [120]:
am_up_out_put = up_and_out(s,k,tau,r,sigma,2000,h, opt_type='p', am=True)
am_put = binomial_tree(s,k,tau,r,sigma,N=2000,opt_type='p',am=True)
'American Up In Put: ${}'.format(round(am_put - am_up_out_put,5))

'American Up In Put: $0.42314'